In [ ]:
#| hide
from dockr.core import *
from dockr.compose import *

# dockr

> A fastai/nbdev-style wrapper around Docker Engine

## Install

```sh
pip install dockr
```

## How to use

### Generate a Dockerfile

In [ ]:
from dockr.core import *

df = (Dockerfile('python:3.11-slim')
    .run('pip install flask')
    .copy('.', '/app')
    .workdir('/app')
    .expose(5000)
    .cmd(['python', 'app.py']))

print(df)

### Build, run, and test (requires Docker daemon)

```python
img = build(df, tag="myapp:latest")          # build image
res = test(img, "python -c \"import flask\"")  # test a command
run(img, detach=True, ports={5000: 5000})     # run container
```

### Generate a docker-compose.yml

In [ ]:
from dockr.compose import *

dc = (Compose()
    .svc('web', image='nginx', ports={80: 80})
    .svc('redis', image='redis:alpine')
    .svc('db', image='postgres:15',
         env={'POSTGRES_PASSWORD': 'secret'},
         volumes={'pgdata': '/var/lib/postgresql/data'})
    .network('backend')
    .volume('pgdata'))

print(dc)

### Orchestrate (requires Docker daemon)

## Key Features

- **Fluent API**: Chain methods to build Dockerfiles and compose files
- **Type-Safe**: Python's type system catches errors early
- **Reverse Proxy**: Built-in Caddy and SWAG support with automatic HTTPS
- **Production Ready**: Includes health checks, restart policies, and security
- **Podman Support**: Works with both Docker and Podman
- **No YAML**: Everything in Python code

## Reverse Proxy with Automatic HTTPS

Deploy production-ready applications with automatic HTTPS using Caddy:

In [ ]:
from dockr.caddy import caddy

dc = (Compose()
    .svc('app', build='.', networks=['web'])
    .svc('caddy', **caddy(
        domain='example.com',
        app='app',
        port=8000,
        dns='cloudflare',  # Automatic HTTPS via DNS
        email='admin@example.com'
    ))
    .network('web')
    .volume('caddy_data')
    .volume('caddy_config'))

dc.up()

## Real-World Examples

Check the [examples/](examples/) directory for complete deployment scenarios:

- **Python Web App**: FastAPI/Flask with PostgreSQL
- **Microservices**: Multi-service architecture with API gateway
- **Static Sites**: nginx-based sites and SPAs (React/Vue)

Each example includes a complete `deploy.py` script and README.

## Documentation

- [Deployment Verification](DEPLOYMENT_VERIFICATION.md) - Comprehensive capability overview
- [Examples](examples/) - Real-world deployment examples
- [API Documentation](https://Karthik777.github.io/dockr/) - Full API reference

```python
dc.save()    # writes docker-compose.yml
dc.up()      # runs docker compose up -d
dc.down()    # runs docker compose down
```